In [6]:
import findspark
findspark.init('/home/joadmin/spark-2.1.2-bin-hadoop2.7/')
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('gdelt_1').config("spark.jars.packages", "graphframes:graphframes:0.5.0-spark2.1-s_2.11").getOrCreate()
csvpathOpt = input("Enter the file path of gdelt_csv:<1>gdelt_small_data.csv <2>20160218230000.export.CSV ")
if(csvpathOpt == '1'):
    csvpath = "/home/joadmin/gdelt_small_data.csv"
    df = spark.read.load(csvpath,format="csv", delimiter="\t", header=True)
elif(csvpathOpt == '2'):
    csvpath = "/home/joadmin/20160218230000.export.CSV"
    df = spark.read.load(csvpath,format="csv", delimiter="\t", header=True)
else:
    print("Input error, faild to load CSV!")
    #break
print(csvpath)

Enter the file path of gdelt_csv:<1>gdelt_small_data.csv <2>20160218230000.export.CSV 2
/home/joadmin/20160218230000.export.CSV


In [7]:
df = spark.read.load(csvpath,format="csv", delimiter="\t", header=True)
#df2 = df.select('Actor1Code AS "f1", Actor2Code as f2, EventID as f3")
#df.printSchema() #print out the schema of your dataset
df2 = df.select([c for c in df.columns if c in {'Actor1Code','Actor1Name','Actor1EthnicCode','Actor1CountryCode'}])
df3 = df.select([c for c in df.columns if c in {'Actor2Code','Actor2Name','Actor2EthnicCode','Actor2CountryCode'}])

df2.join(df3)

vertex = df2.distinct() #get distinct row
vertex = vertex.selectExpr("Actor1Name as id", "Actor1Code as Actor1Code", "Actor1EthnicCode as Actor1EthnicCode", "Actor1CountryCode as Actor1CountryCode" )
vertex.show() #print out the dataset

#df2.filter(col('Actor1Code').isin(['MIL'])).show()


+--------------------+------------+----------------+-----------------+
|                  id|  Actor1Code|Actor1EthnicCode|Actor1CountryCode|
+--------------------+------------+----------------+-----------------+
|             LEBANON|         LBN|            null|              LBN|
|            MICHIGAN|         USA|            null|              USA|
|             NIGERIA|      NGAJUD|            null|              NGA|
|            BELIEVER|         REL|            null|             null|
|       CIVIL SOCIETY|         CVL|            null|             null|
|               SYRIA|      SYRMIL|            null|              SYR|
|               ODESA|      UKRGOV|            null|              UKR|
|            PRODUCER|         BUS|            null|             null|
|               CHINA|      CHNGOV|            null|              CHN|
|              LONDON|      GBRHLH|            null|              GBR|
|             TUNISIA|         TUN|            null|              TUN|
|     

In [9]:
df.createOrReplaceTempView("table1")
edge = spark.sql("SELECT Actor1Name AS src, Actor2Name as dst, EventCode as relationship from table1")
edge.collect()

edge.show()


+-------------+-------------+------------+
|          src|          dst|relationship|
+-------------+-------------+------------+
|         null|      TROOPER|         173|
|         null|         NAVY|         192|
|         null|         NAVY|         192|
|     BUSINESS|         null|         172|
|      TROOPER|UNITED STATES|         173|
|DAVID CAMERON|      BRITISH|         040|
|  AUTHORITIES|         null|         172|
|         IRAN|  SOUTH KOREA|         150|
|         IRAN|  SOUTH KOREA|         150|
|         IRAN|     MILITARY|         046|
|         IRAN|     MILITARY|         046|
| SOUTH KOREAN|  NEWS AGENCY|         010|
|       MEXICO|     CONGRESS|         020|
|     MILITARY|         IRAN|         046|
|     MILITARY|         IRAN|         046|
|     MOTOROLA|         null|         010|
|    TERRORIST|     PAKISTAN|         190|
|UNITED STATES|         null|         015|
|   WASHINGTON|  SOUTH KOREA|         150|
|         null|        CUBAN|         173|
+----------

In [18]:
from graphframes import *
#--packages graphframes:graphframes:0.5.0-spark2.1-s_2.11

g = GraphFrame(vertex, edge)

# Query: Get in-degree of each vertex.
g.inDegrees.show()
#print (type(g.inDegrees))
#print (g.inDegrees.orderBy(g.inDegrees.inDegree.desc()).collect())
g.edges.filter("EventCode = '173'").count()

+-------------------+--------+
|                 id|inDegree|
+-------------------+--------+
|             HUNTER|       1|
|             SENATE|      11|
|          SPOKESMAN|       2|
|               GANG|       3|
|   PRANAB MUKHERJEE|       3|
|          DISSIDENT|       1|
|WORLD WILDLIFE FUND|       1|
|         NEW JERSEY|       7|
|   OPPOSITION PARTY|       1|
|           UPRISING|       3|
|            ORLANDO|       5|
|           CAMBODIA|      11|
|           COMMANDO|       1|
|              JAPAN|       4|
|             MURRAY|       1|
|         BANGLADESH|       6|
|             UGANDA|       1|
|       INDIANAPOLIS|       5|
|    FIRE DEPARTMENT|       1|
| PARAMILITARY FORCE|       1|
+-------------------+--------+
only showing top 20 rows

<class 'pyspark.sql.dataframe.DataFrame'>
[Row(id=None, inDegree=605), Row(id='UNITED STATES', inDegree=217), Row(id='CUBA', inDegree=62), Row(id='UNITED KINGDOM', inDegree=35), Row(id='SYRIA', inDegree=35), Row(id='GOVERNMENT', i

95

In [36]:
# Run PageRank algorithm, and show results.
results = g.pageRank(resetProbability=0.15, maxIter=10)
results.vertices.select("id", "pagerank").show()

+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|                BANK| 0.1965300521588072|
|            VILLAGER|               0.15|
|            RICHMOND|               0.15|
|             ONTARIO|0.17756826372808115|
|CHURCH OF THE LIV...| 0.3009865444309504|
|                MONK|               0.15|
|        NEIGHBORHOOD|0.17576402256189974|
|             IRELAND| 0.8238602150982135|
|               SAMOA| 1.7868071402492123|
|PROSECUTING ATTORNEY|               0.15|
|               GHANA|               0.15|
|               GHANA|               0.15|
|            INTERPOL|               0.15|
|          FAISALABAD|               0.15|
|        JACKSONVILLE| 0.6385133040794575|
|               BENIN|0.26415095303346814|
|              FRANCE| 0.8981574513191302|
|            GHANAIAN|               0.15|
|            GHANAIAN|               0.15|
|            COLUMBUS|               0.15|
+----------